In [0]:
!pip install q keras==2.3.0
!pip install tensorflow_hub

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt

#%tensorflow_version 1.x
import tensorflow as tf

import tensorflow_hub as hub
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import pickle

import keras
keras.__version__

Using TensorFlow backend.


'2.3.0'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_root='/content/drive/My Drive/Audio_Quality_Analysis/data/dataset_1/train'
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)

datagen_kwargs = dict(rescale=1./255, validation_split=.20)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory( TRAINING_DATA_DIR, subset="validation",
                                                      shuffle=True, target_size=IMAGE_SHAPE)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(TRAINING_DATA_DIR, subset="training",
                                                      shuffle=True, target_size=IMAGE_SHAPE)

Found 2240 images belonging to 2 classes.
Found 8963 images belonging to 2 classes.


In [0]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)

dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
print(dataset_labels)
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 2)
[('good', 0), ('noise', 1)]
['Good' 'Noise']


In [0]:
le = LabelEncoder()
labels = le.fit_transform(dataset_labels)
#labels = np_utils.to_categorical(labels, 3)
print(labels)
with open('/content/drive/My Drive/Audio_Quality_Analysis/model/labelEncoderFile_04222020_v1.pkl', 'wb') as fid:
    pickle.dump(le, fid)

[0 1]


In [0]:
train_generator.class_indices

{'good': 0, 'noise': 1}

In [0]:
model = tf.keras.Sequential([
                    hub.KerasLayer(
                        "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                        output_shape=[1280], trainable=False
                    ),
                    tf.keras.layers.Dropout(0.4),
                    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
                    ])

model.build([None, 224, 224, 3])
model.summary()

model.compile( optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit_generator( train_generator, epochs=10, verbose=1,
                            steps_per_epoch=steps_per_epoch,
                            validation_data=valid_generator,
                            validation_steps=val_steps_per_epoch
                           ).history

model.save("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQuality_MobileNet_TL_GD_V1_04222.h5")
model.save_weights("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQualityWeights_MobileNet_TL_GD_V1_0422.h5")

Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/10
281/281 [==============================] - 8042s 29s/step - loss: 0.4152 - acc: 0.8065 - val_loss: 0.4464 - val_acc: 0.7817
Epoch 2/10
281/281 [==============================] - 47s 168ms/step - loss: 0.2864 - acc: 0.8772 - val_loss: 0.3895 - val_acc: 0.8067
Epoch 3/10
281/281 [==============================] - 46s 165ms/step - loss: 0.2547 - acc: 0.8936 - val_loss: 0.3743 - val_acc: 0.8219
Epoch 4/10
281/281 [==============================] - 47s 166ms/step - loss: 0.2478 - acc: 0.8974 - val_loss: 0.3806 - val_acc: 0.8165
Epoch 5/10
281/281 [==============================] - 46s 164ms/step - loss: 0.2374 - acc: 0.8995 - val_loss: 0.3971 - val_acc: 0.8058
Epoch 6/10
281/281 [==============================] - 46s 165ms/step - loss: 0.2368 - acc: 0.9019 - val_loss: 0.4533 - val_acc: 0.7937
Epoch 7/10
281/281 [==============================] - 46s 164ms/step - loss: 0.2376 - acc: 0.8995 - val_loss: 0.3884 - val_acc: 0.8170
Epoch 8/10
281/281 [==============================] - 4

In [0]:
from keras import models
model = tf.keras.models.load_model('/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQuality_TL_GD_V1_04222.h5',custom_objects={'KerasLayer':hub.KerasLayer})
model.load_weights('/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQualityWeights_TL_GD_V1_0422.h5')
model.summary()

In [0]:
val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)

print("Validation batch shape:", val_image_batch.shape)

tf_model_predictions = model.predict(val_image_batch)
tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
tf_pred_dataframe.columns = dataset_labels

print("Prediction results for the first elements")
tf_pred_dataframe.head()